In [8]:
from policy import PolicyFactory, ViolationMode


In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [21]:
policy = PolicyFactory.with_tolerance(over_tol=20.5,
                                      under_tol=.5,
                                      violation=ViolationMode.ERROR)

target = 0
x = np.linspace(-10, 10, 100)
plt.plot(x, [policy.cost(target, i) for i in x])

ConstraintViolation: Нарушено ограничение политики